<img alt="Colaboratory logo" width="15%" src="https://raw.githubusercontent.com/carlosfab/escola-data-science/master/img/novo_logo_bg_claro.png">

#### **Data Science na Prática 3.0**
*by [sigmoidal.ai](https://sigmoidal.ai)*

---


# Prevendo preços de imóveis com Redes Neurais

Para demonstrar como as redes neurais podem ser usadas para um problema de regressão, para prever preços de venda de imóveis, vou usar o conhecido *dataset* California housing.

<center><img src="https://www.wartsila.com/images/default-source/default-album/california_housing.tmb-1920x690.jpg?sfvrsn=126bc44_1" width="80%"></center>



A maneira que iremos construir a arquitetura de redes neurais é por meio da API `Sequential` do `keras`.


In [1]:
# importar as bibliotecas necessárias
%tensorflow_version 2.x
import tensorflow as tf
from tensorflow import keras
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd

Colab only includes TensorFlow 2.x; %tensorflow_version has no effect.


Uma vez importados todos os pacotes necessários, é hora de carregarmos os dados e começarmos o nosso projeto!

Para isso, utilizaremos um dataset do próprio `sklearn.dataset`, o `California Housing`.

In [2]:
# baixar os dados
from sklearn.datasets import fetch_california_housing
housing = fetch_california_housing()
feature_names = housing['feature_names']
feature_names.append('Target')

Além disso, precisamos transformar esse conjunto em um DataFrame dentro do Pandas.

Podemos fazer isso com a sequência de código abaixo:

In [3]:
# converter dados em DataFrame
df = pd.concat([pd.DataFrame(housing.data), pd.Series(housing.target)], axis=1)
df.columns = feature_names

# ver as 5 primeiras entradas
df.head()

,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude,Target
0,8.3252,41.0,6.984127,1.023810,322.0,2.555556,37.88,-122.23,4.526
1,8.3014,21.0,6.238137,0.971880,2401.0,2.109842,37.86,-122.22,3.585
2,7.2574,52.0,8.288136,1.073446,496.0,2.802260,37.85,-122.24,3.521
3,5.6431,52.0,5.817352,1.073059,558.0,2.547945,37.85,-122.25,3.413
4,3.8462,52.0,6.281853,1.081081,565.0,2.181467,37.85,-122.25,3.422


Agora sim estamos prontos para, de fato, fazer nossas previsões usando redes neurais.

Abaixo, podemos ver o código completo, desde a separaç!ão dos dados até o resultado do modelo.


In [4]:
# 1. escolher e importar um modelo
# Rede Neural usando API SEQUENTIAL

# 3. Separar os dados entre feature matrix e target vector
X = df.drop('Target', axis=1)
y = df['Target']

# 3.1 Dividir o dataset entre treino e teste
X_train_completo, X_test, y_train_completo, y_test = train_test_split(X, y)

# 3.2 Dividir os dados de treino entre treino e validação
X_train, X_valid, y_train, y_valid = train_test_split(X_train_completo, y_train_completo)

# 3.3 Padronizar os dados (MUITO importante para redes neurais)
sc = StandardScaler().fit(X_train)
X_train = sc.transform(X_train)
X_valid = sc.transform(X_valid)
X_test = sc.transform(X_test)

# 2. Instanciar e escolher os hyperparameters
model = keras.models.Sequential()
model.add(keras.layers.Dense(50, input_shape=(X_train.shape[1:]), activation='relu'))
model.add(keras.layers.Dense(10, activation='relu'))
model.add(keras.layers.Dense(1))

# 4. Fit do modelo (treinar)
model.compile(optimizer='sgd', loss='mean_squared_error')
history = model.fit(X_train, y_train, epochs=25, validation_data=(X_valid, y_valid))

# 5. Avaliar o modelo
print(model.evaluate(X_test, y_test))

# 5.1 Fazer previsões em cima de novos dados
y_pred = model.predict(X_test)

Epoch 1/25
363/363 [==============================] - 7s 6ms/step - loss: 0.8027 - val_loss: 0.4808
Epoch 2/25
363/363 [==============================] - 1s 3ms/step - loss: 0.4662 - val_loss: 0.4272
Epoch 3/25
363/363 [==============================] - 1s 3ms/step - loss: 0.4388 - val_loss: 0.4163
Epoch 4/25
363/363 [==============================] - 1s 3ms/step - loss: 0.4115 - val_loss: 0.3945
Epoch 5/25
363/363 [==============================] - 1s 3ms/step - loss: 0.3970 - val_loss: 0.3870
Epoch 6/25
363/363 [==============================] - 1s 3ms/step - loss: 0.3870 - val_loss: 0.3807
Epoch 7/25
363/363 [==============================] - 1s 3ms/step - loss: 0.3789 - val_loss: 0.3776
Epoch 8/25
363/363 [==============================] - 1s 3ms/step - loss: 0.3708 - val_loss: 0.3737
Epoch 9/25
363/363 [==============================] - 1s 3ms/step - loss: 0.3669 - val_loss: 0.3657
Epoch 10/25
363/363 [==============================] - 1s 3ms/step - loss: 0.3608 - val_loss: 0.3598

In [5]:
# ATENÇÃO para o shape do input
print(X_train.shape[1])
print(X_train.shape[1:])

8
(8,)
